In [1]:
import pandas as pd
import numpy as np
from datasets import load_dataset
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
import re

C:\Users\wreep\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\wreep\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
from urllib.request import urlretrieve
import pandas as pd

# Download the parquet table
table_url = f'https://huggingface.co/datasets/poloclub/diffusiondb/resolve/main/metadata.parquet'
urlretrieve(table_url, 'metadata.parquet')

# Read the table using Pandas
df = pd.read_parquet('metadata.parquet')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 13 columns):
 #   Column       Dtype              
---  ------       -----              
 0   image_name   object             
 1   prompt       object             
 2   part_id      uint16             
 3   seed         uint32             
 4   step         uint16             
 5   cfg          float32            
 6   sampler      uint8              
 7   width        uint16             
 8   height       uint16             
 9   user_name    object             
 10  timestamp    datetime64[us, UTC]
 11  image_nsfw   float32            
 12  prompt_nsfw  float32            
dtypes: datetime64[us, UTC](1), float32(3), object(3), uint16(4), uint32(1), uint8(1)
memory usage: 108.7+ MB


In [4]:
df.head()

,image_name,prompt,part_id,seed,step,cfg,sampler,width,height,user_name,timestamp,image_nsfw,prompt_nsfw
0,2217ccbd-a1c6-47ac-9a2d-79649727c834.png,"a portrait of a female robot made from code, v...",1,2615452544,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 11:56:00+00:00,0.554853,0.001621
1,c78807b7-d55a-4a2d-a6b6-9192b18941ad.png,a portrait of a female robot made from a cloud...,1,4231506957,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 11:59:00+00:00,0.153645,0.000707
2,dc71658a-5e4b-4dca-861a-e1535510348b.png,"only memories remain, trending on artstation",1,544607824,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 12:02:00+00:00,0.062496,0.000425
3,48eb7e17-a3cf-4eb8-96a9-d8e3e23fa1af.png,dream swimming pool with nobody,1,3500509234,50,7.0,8,512,512,019084defdd44d2dfcd200bf7aaae53034263168e10d6b...,2022-08-20 07:22:00+00:00,0.030799,0.003586
4,601d9792-eccd-4850-97a7-edbe91d3464c.png,a dog doing weights. epic oil painting.,1,1312381211,50,12.0,8,512,768,03cb377ebf364ea2761b5710625974c0d2d25285fec982...,2022-08-20 12:28:00+00:00,0.181035,0.030822


In [5]:
prompts = df[4000:5000]

In [6]:
prompts

,image_name,prompt,part_id,seed,step,cfg,sampler,width,height,user_name,timestamp,image_nsfw,prompt_nsfw
4000,8d3dfd5a-e3ac-4035-9f4f-48a13b71c1f6.png,photorealistic rendering of eldritch demon sta...,5,66123366,50,7.0,8,512,512,02251bf38dbe2d0ca8a9eb878ccf8d50e69a02568e03f1...,2022-08-18 21:51:00+00:00,0.154185,0.000182
4001,33c883ef-5296-4777-bfe5-1e300524dfe2.png,a white aurochs with gold horns charging throu...,5,94091301,50,7.0,8,512,512,024042b48501d126881d6e2aaf524ad1261a3634859bde...,2022-08-18 17:26:00+00:00,0.108092,0.027852
4002,f4a4072e-acd7-4b57-8920-bd66811e678a.png,a woman looking over her shoulder dramatically...,5,772375564,50,7.0,8,512,512,024042b48501d126881d6e2aaf524ad1261a3634859bde...,2022-08-18 19:38:00+00:00,0.225592,0.013805
4003,38f333e1-02ec-4189-a2a5-2f287ba68602.png,a woman looking over her shoulder dramatically...,5,92445970,50,7.0,8,512,512,024042b48501d126881d6e2aaf524ad1261a3634859bde...,2022-08-18 19:39:00+00:00,0.154625,0.016430
4004,e18b653f-f7c1-48f7-b4e3-2e3f59e3f757.png,"a purple ocean with a bizarre alien caldera, s...",5,2137055400,50,7.0,8,512,512,024042b48501d126881d6e2aaf524ad1261a3634859bde...,2022-08-18 22:20:00+00:00,0.050091,0.004102
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,42c9d8cc-9aff-4609-9f31-d273e2f5ef71.png,scarlet witch outfit concept art trending on a...,5,3750811694,50,7.0,8,512,640,ffee0f4b6fd0fa325e9b53fab6f2098ff2c0414e6e0d99...,2022-08-19 03:46:00+00:00,0.552977,0.001335
4996,2aacf96e-4c92-45b2-bed3-62374bc918d3.png,[ angel - like ]!! scarlet witch outfit concep...,5,1111992836,50,7.0,8,512,640,ffee0f4b6fd0fa325e9b53fab6f2098ff2c0414e6e0d99...,2022-08-19 03:49:00+00:00,0.824537,0.000505
4997,a6baf9ae-68d2-4517-afa7-0a6b4ec21a3f.png,[ obscenely corrupted dismally heathen - like ...,5,1344582638,50,7.0,8,512,640,ffee0f4b6fd0fa325e9b53fab6f2098ff2c0414e6e0d99...,2022-08-19 03:54:00+00:00,0.394097,0.033509
4998,c7f83edb-3771-4ee6-9d0d-605fe45644c6.png,elizabeth olsen in a [ tenaciously witch - lik...,5,2755046587,50,7.0,8,512,640,ffee0f4b6fd0fa325e9b53fab6f2098ff2c0414e6e0d99...,2022-08-19 03:59:00+00:00,0.141651,0.098639


In [ ]:
for idx, row in prompts.iterrows():
    Classprompt = row['prompt']
    Numprompt = row['prompt']
    for word in re.findall(r'\b\w+\b', row['prompt']):
        if word not in df['Word'].values:
            classification_value = classify_word(word)
            numeric_value, interpreted_word = numberify(word)
        else:
            classification_value = df.loc[df['Word'] == word, 'Classification'].values[0]
            numeric_value = df.loc[df['Word'] == word, 'Number'].values[0]
        Classprompt = Classprompt.replace(word, str(classification_value))
        Numprompt = Numprompt.replace(word, str(numeric_value))
    prompts.at[idx, 'Classprompt'] = int(Classprompt) if Classprompt.isdigit() else Classprompt
    prompts.at[idx, 'Numprompt'] = int(Numprompt) if Numprompt.isdigit() else Numprompt

KeyError: 'Word'

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X, y)